In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
import keras.datasets.mnist as mnist
(train_image, train_label), (test_image, test_label) = mnist.load_data() 

In [3]:
train_image = np.expand_dims(train_image, axis = -1)
test_image = np.expand_dims(test_image, axis = -1)

# 原始网络搭建：每层起个名字

In [9]:
model = keras.Sequential()

In [10]:
from keras import layers

In [11]:
# 用name参数给需要的层起名：
# 卷积池化层：
model.add( layers.Conv2D( filters=64, kernel_size=(3,3), activation = 'relu', input_shape=(28,28,1), name = 'gby_c1' ) )  
model.add( layers.Conv2D( filters=64, kernel_size=(3,3), activation='relu', name = 'gby_c2') )
model.add( layers.MaxPooling2D()  )  # 该层没可训练参数，也就没权重
# 进入全连接层：
model.add( layers.Flatten() )        # 该层没可训练参数，也就没权重
model.add( layers.Dense(256, activation='relu', name = 'no_d1' ))
model.add( layers.Dropout(0.5) )     # 该层没可训练参数，也就没权重
model.add(layers.Dense(10, activation='softmax', name = 'no_d2')) 

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gby_c1 (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
gby_c2 (Conv2D)              (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
no_d1 (Dense)                (None, 256)               2359552   
_________________________________________________________________
no_d2 (Dropout)              (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)               

In [12]:
model.compile( optimizer='adam',
               loss = 'sparse_categorical_crossentropy',  # 顺序编码
               metrics=['acc']
)

In [13]:
model.fit(train_image, train_label, epochs = 2, batch_size = 512, validation_data=(test_image, test_label) )


Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 98s 2ms/step - loss: 2.1186 - acc: 0.8586 - val_loss: 0.0627 - val_acc: 0.9802
Epoch 2/2
60000/60000 [==============================] - 99s 2ms/step - loss: 0.0943 - acc: 0.9715 - val_loss: 0.0516 - val_acc: 0.9832


# 新建层结构模型：用形式3按“层名字”加载部分参数

In [15]:
# 对原模型权重的保存：
model.save_weights('my_model_weights_25.h5')

### 新建一个层结构： 

In [16]:
# 想要载入权重的层，其name和原始模型的name保存一致；不想载入权重的层，其name不要和原模型一样！
model_new = keras.Sequential()

# 卷积池化层：载入权重 √
model_new.add( layers.Conv2D( filters=64, kernel_size=(3,3), activation = 'relu', input_shape=(28,28,1), name = 'gby_c1' ) )  
model_new.add( layers.Conv2D( filters=64, kernel_size=(3,3), activation='relu', name = 'gby_c2') )
model_new.add( layers.MaxPooling2D()  )  # 该层没可训练参数，也就没权重
# 进入全连接层：不载入 ×
model_new.add( layers.Flatten() )        # 该层没可训练参数，也就没权重
model_new.add( layers.Dense(256, activation='relu', name = 'no_d11' ))
model_new.add( layers.Dropout(0.5) )     # 该层没可训练参数，也就没权重
model_new.add(layers.Dense(10, activation='softmax', name = 'no_d22')) 

In [17]:
model_new.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gby_c1 (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
gby_c2 (Conv2D)              (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 9216)              0         
_________________________________________________________________
no_d11 (Dense)               (None, 256)               2359552   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
no_d22 (Dense)               (None, 10)               

In [19]:
# 不能忘记要“编译一下”：
model_new.compile( optimizer='adam',
                   loss = 'sparse_categorical_crossentropy',  # 顺序编码
                   metrics=['acc']
)

### 载入部分层的权重：卷积层载，全连接层不载 

In [20]:
# 多用一个参数设定：byname = True —— 新/老模型，名字相同的层，可以载入参数
model_new.load_weights( 'my_model_weights_25.h5', by_name=True )

In [21]:
# 现在的模型已经功能完全具备，可以测试一下它的预测功能：
model_new.evaluate(test_image, test_label)

10000/10000 [==============================] - 4s 386us/step


[3.4070851642608644, 0.061799999326467514]

说明：载入的卷积层的参数后，为什么预测结果还是不好呢？

因为：卷积层是用来特征提取的，把卷积层的参数载入，可以使得模型的特征提取的非常好。但是全连接层是进行“**特征汇聚**”与最后的“**结果转化**”，所以如果全连接层参数不载入，说明当前的模型就是“**临门一脚**”！特征提取的很好，但是不总结，最后的结果依然不好。

启示：迁移学习，我们可以选只导入卷积层参数，最后的Dense全连接层参数很好训练，再单独训练即可。—— 卷积层参数 与 全连接层参数，二者的关系应该是“**相对比较独立**”的：因为卷积层就是辅助特征提取，它的各种权重参数就是它特征提取的方式。因此如果别人已经训练好了一个特征提取能力非常好的网络，我们可以就载入它的“卷积层参数”，后面相对独立的“全连接参数”我们可以根据自己的需求来做“**修改与再训练**”。